# Phobert label

In [1]:
import torch
import pandas as pd
from transformers import RobertaForSequenceClassification, AutoTokenizer

model = RobertaForSequenceClassification.from_pretrained("wonrax/phobert-base-vietnamese-sentiment")

tokenizer = AutoTokenizer.from_pretrained("wonrax/phobert-base-vietnamese-sentiment", use_fast=False)


d:\Study file\Fall 2024 - Capstone\Capstone\.conda\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
d:\Study file\Fall 2024 - Capstone\Capstone\.conda\Lib\site-packages\huggingface_hub\file_download.py:139: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Minh Truc\.cache\huggingface\hub\models--wonrax--phobert-base-vietnamese-sentiment. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run 

[[0.001841121818870306, 0.988153874874115, 0.01000502984970808]]


In [3]:
data = pd.read_csv('sentiment.csv')

In [4]:
data

,Text
0,Điều ít biết về sàn TMĐT Temu đang gây bão vì ...
1,Margin cao kỷ lục nhưng thanh khoản thị trường...
2,Tự doanh CTCK trở lại mua ròng trong phiên 24/...
3,Tỷ phú Nguyễn Đăng Quang chỉ nắm 18 cổ phiếu M...
4,Chuỗi siêu thị của tỉ phú Nguyễn Đăng Quang bắ...
...,...
10474,Chứng khoán: Đà phục hồi khó kéo dài - Dù thị ...
10475,Công ty chứng khoán: Rủi ro nào từ nghiệp vụ c...
10476,VNDIRECT công bố Báo cáo Chiến lược đầu tư Việ...
10477,Lịch chốt quyền cổ tức tuần 5/12 - 9/12 - Tron...


In [5]:
def predict_sentiment(text):
    try:
        # Word-segmented text (nếu chưa phân đoạn từ trước, bạn cần phân đoạn)
        input_ids = torch.tensor([tokenizer.encode(text, truncation=True, padding=True)])
        with torch.no_grad():
            outputs = model(input_ids)
            # Softmax để lấy xác suất dự đoán
            probabilities = outputs.logits.softmax(dim=-1).tolist()[0]
            # Xác định nhãn cảm xúc dựa vào xác suất cao nhất
            label = probabilities.index(max(probabilities))
            # Chuyển nhãn thành -1, 0, 1
            if label == 0:   # Negative
                return -1
            elif label == 1: # Positive
                return 1
            else:            # Neutral
                return 0
    except Exception as e:
        print(f"Error in sentiment prediction: {e}")
        return None

# Áp dụng hàm dự đoán sentiment cho cột text của DataFrame
data['label'] = data['Text'].apply(predict_sentiment)


In [6]:

# Lưu DataFrame với cột dự đoán cảm xúc vào file CSV mới
data.to_csv("articles_with_sentiment.csv", index=False, encoding='utf-8')

data

,Text,label
0,Điều ít biết về sàn TMĐT Temu đang gây bão vì ...,1
1,Margin cao kỷ lục nhưng thanh khoản thị trường...,-1
2,Tự doanh CTCK trở lại mua ròng trong phiên 24/...,0
3,Tỷ phú Nguyễn Đăng Quang chỉ nắm 18 cổ phiếu M...,1
4,Chuỗi siêu thị của tỉ phú Nguyễn Đăng Quang bắ...,1
...,...,...
10474,Chứng khoán: Đà phục hồi khó kéo dài - Dù thị ...,1
10475,Công ty chứng khoán: Rủi ro nào từ nghiệp vụ c...,-1
10476,VNDIRECT công bố Báo cáo Chiến lược đầu tư Việ...,1
10477,Lịch chốt quyền cổ tức tuần 5/12 - 9/12 - Tron...,0


# Visobert label

In [7]:
from transformers import pipeline

In [8]:
# Khởi tạo pipeline với model "5CD-AI/Vietnamese-Sentiment-visobert"
model_path = '5CD-AI/Vietnamese-Sentiment-visobert'
sentiment_task = pipeline("sentiment-analysis", model=model_path, tokenizer=model_path)

# Hàm phân tích cảm xúc
def predict_sentiment(text):
    try:
        # Sử dụng pipeline để dự đoán cảm xúc
        result = sentiment_task(text)[0]  # Trả về kết quả đầu tiên trong danh sách kết quả
        label = result['label']
        # Chuyển nhãn thành -1, 0, 1
        if label == 'NEG':
            return -1
        elif label == 'POS':
            return 1
        else:  # Trường hợp 'NEU' (nếu có) hoặc bất kỳ nhãn khác
            return 0
    except Exception as e:
        print(f"Error in sentiment prediction: {e}")
        return None

# Đọc dữ liệu từ file CSV đã làm sạch
data = pd.read_csv("sentiment.csv")

# Áp dụng hàm dự đoán sentiment cho cột text của DataFrame
data['label'] = data['Text'].apply(predict_sentiment)

# Lưu DataFrame với cột dự đoán cảm xúc vào file CSV mới
data.to_csv("articles_with_sentiment_visobert.csv", index=False, encoding='utf-8')

data

d:\Study file\Fall 2024 - Capstone\Capstone\.conda\Lib\site-packages\huggingface_hub\file_download.py:139: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Minh Truc\.cache\huggingface\hub\models--5CD-AI--Vietnamese-Sentiment-visobert. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


,Text,label
0,Điều ít biết về sàn TMĐT Temu đang gây bão vì ...,1
1,Margin cao kỷ lục nhưng thanh khoản thị trường...,-1
2,Tự doanh CTCK trở lại mua ròng trong phiên 24/...,0
3,Tỷ phú Nguyễn Đăng Quang chỉ nắm 18 cổ phiếu M...,0
4,Chuỗi siêu thị của tỉ phú Nguyễn Đăng Quang bắ...,-1
...,...,...
10474,Chứng khoán: Đà phục hồi khó kéo dài - Dù thị ...,-1
10475,Công ty chứng khoán: Rủi ro nào từ nghiệp vụ c...,-1
10476,VNDIRECT công bố Báo cáo Chiến lược đầu tư Việ...,0
10477,Lịch chốt quyền cổ tức tuần 5/12 - 9/12 - Tron...,0


# Mixing

Tiêu đề có ngữ cảnh tích cực (vd giá vàng tăng) có thật sự ảnh hưởng tích cực lên thị trường ckps không? (vàng tăng -> bán ck mua vàng -> dự đoán xu hướng ckps giảm) Như vậy cần phải trả lời với thông tin như này thì ảnh hưởng như thế nào lên thị trường ckps.

Prompt gpt: Với thông tin hợp đồng tương lai VN30 là sản phẩm phái sinh dựa trên chỉ số VN30, bao gồm 30 cổ phiếu có vốn hóa lớn và thanh khoản cao nhất trên sàn HOSE. Nhà đầu tư tham gia cam kết mua hoặc bán chỉ số này tại một mức giá đã xác định vào một thời điểm trong tương lai, sử dụng đòn bẩy để giao dịch với số vốn ban đầu thấp hơn giá trị hợp đồng. Điểm nổi bật của hợp đồng tương lai VN30 là tính thanh khoản cao và khả năng đầu tư hai chiều, cho phép nhà đầu tư kiếm lợi nhuận khi thị trường tăng (long) hoặc giảm (short). Ngoài ra, nó còn là công cụ phòng ngừa rủi ro hiệu quả, giúp bảo vệ danh mục đầu tư trước biến động của thị trường cơ sở. Đánh giá đoạn dữ liệu này:f'{context} có ảnh hưởng như thế nào lên hợp đồng tương lai vn30, nếu cảm thấy thông tin này giúp hợp đồng tăng giá thì trả lời số '1', ngược lại trả lời '-1', còn nếu thông tin không ảnh hưởng gì tới hợp đồng thì trả lời số '0'.

In [10]:
pbert = pd.read_csv('articles_with_sentiment.csv')
vbert = pd.read_csv('articles_with_sentiment_visobert.csv')

In [11]:
pbert

,Text,label
0,Điều ít biết về sàn TMĐT Temu đang gây bão vì ...,1
1,Margin cao kỷ lục nhưng thanh khoản thị trường...,-1
2,Tự doanh CTCK trở lại mua ròng trong phiên 24/...,0
3,Tỷ phú Nguyễn Đăng Quang chỉ nắm 18 cổ phiếu M...,1
4,Chuỗi siêu thị của tỉ phú Nguyễn Đăng Quang bắ...,1
...,...,...
10474,Chứng khoán: Đà phục hồi khó kéo dài - Dù thị ...,1
10475,Công ty chứng khoán: Rủi ro nào từ nghiệp vụ c...,-1
10476,VNDIRECT công bố Báo cáo Chiến lược đầu tư Việ...,1
10477,Lịch chốt quyền cổ tức tuần 5/12 - 9/12 - Tron...,0


In [12]:
vbert

,Text,label
0,Điều ít biết về sàn TMĐT Temu đang gây bão vì ...,1
1,Margin cao kỷ lục nhưng thanh khoản thị trường...,-1
2,Tự doanh CTCK trở lại mua ròng trong phiên 24/...,0
3,Tỷ phú Nguyễn Đăng Quang chỉ nắm 18 cổ phiếu M...,0
4,Chuỗi siêu thị của tỉ phú Nguyễn Đăng Quang bắ...,-1
...,...,...
10474,Chứng khoán: Đà phục hồi khó kéo dài - Dù thị ...,-1
10475,Công ty chứng khoán: Rủi ro nào từ nghiệp vụ c...,-1
10476,VNDIRECT công bố Báo cáo Chiến lược đầu tư Việ...,0
10477,Lịch chốt quyền cổ tức tuần 5/12 - 9/12 - Tron...,0


In [18]:
pdiff = pbert[pbert['label'] != vbert['label']]
vdiff = vbert[vbert['label'] != pbert['label']]
same = pbert[pbert['label'] == vbert['label']]

In [22]:
pdiff.to_csv('pdiff.csv', index=False)
vdiff.to_csv('vdiff.csv', index=False)
same.to_csv('same.csv', index=False)